In [1]:
import sqlite3

from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    select,
    text

)

from sqlalchemy.orm import sessionmaker

import pandas as pd     

In [25]:
df = pd.read_json('catalog.json').T

In [18]:
print(df['coi_data'][0:2])
dict = df['coi_data'][0:2].to_dict()
print(dict)
print(df.columns)

SWA101    {'course_coi': 1.7000000000000002, 'course_lev...
SWA102    {'course_coi': 1.5, 'course_level_coi': 0.0, '...
Name: coi_data, dtype: object
{'SWA101': {'course_coi': 1.7000000000000002, 'course_level_coi': 0.0, 'curric_coi': 1.6, 'percent_in_range': 62.2}, 'SWA102': {'course_coi': 1.5, 'course_level_coi': 0.0, 'curric_coi': 1.6, 'percent_in_range': 62.2}}
Index(['coi_data', 'concurrent_courses', 'course_campus', 'course_credits',
       'course_description', 'course_id', 'course_offered', 'course_title',
       'department_abbrev', 'gpa_distro', 'is_bottleneck', 'is_gateway',
       'prereq_graph', 'prereq_string'],
      dtype='object')


In [ ]:
dropped_df = df.drop(columns=['is_gateway', 'prereq_graph', 'coi_data', ''])

SWA101      {'x': {'edges': {'from': {'8590': 'SWA 101'}, ...
SWA102      {'x': {'edges': {'from': {'8590': 'SWA 101', '...
SWA103      {'x': {'edges': {'from': {'8591': 'SWA 102', '...
SWA134      {'x': {'edges': {'from': {'8593': 'SWA 134'}, ...
SWA201      {'x': {'edges': {'from': {'8592': 'SWA 103', '...
                                  ...                        
HSERV482                                                 None
HSERV488                                                 None
HSERV490                                                 None
HSERV493                                                 None
HSERV499                                                 None
Name: prereq_graph, Length: 5777, dtype: object


In [ ]:
#course offered contains values outside of seasons

print(df['course_offered'].nunique())

589


In [ ]:
#filter out values that are not seasons
seasons = ['S.', 'A.', 'W.', 'Sp.']

df = df[df['course_offered'].isin(seasons)]


4


In [27]:
#confirm class id is unique
print(len(df))
print(df['course_id'].nunique())

5777
5777


In [30]:
#create mean gpa

gpa_data = pd.concat([df['course_id'], df['gpa_distro'].apply(pd.Series)], axis=1)
print(gpa_data.columns)


Index(['course_id',           0,           1,           2,           3,
                 4,           5,           6,           7,           8,
                 9,          10,          11,          12,          13,
                14,          15,          16,          17,          18,
                19,          20,          21,          22,          23,
                24,          25,          26,          27,          28,
                29,          30,          31,          32,          33,
                34,          35,          36,          37,          38,
                39,          40],
      dtype='object')
